#**STORIES**
###Made by: (Y0z64)[https://github.com/Y0z64]

##Instructions
- Run the first cell in **Requirements** and wait for it to display "Requirements correctly installed"
- Run the second cell and wait for it to load
- Run the last cell under **Start app**
- When the app finishes loading CNTR+click the public link or use locally in colab

##Getting started
- Write start in the input and wait for a prompt to appear
- The space under the chat shows recommended actions to take, to make an action just write it in the input box
- Under the recommended action the image for the actual propmt will be displayed

##Knwon issues
- If after sometime the image generation gets too slow, close the window and re-run the last cell
- Sometimes the prompt generation can get repetitive or incoherent, those are quirks of the generation, if it gets too bad close window and re-run last cell

In [ ]:
#@markdown ###Check if there is conected gpu
from IPython.display import clear_output

try:
  !nvidia-smi
  clear_output()
  print("Gpu running correctly")
except:
  print("No gpu connected")


Gpu running correctly


##**Requirements**

In [ ]:
!pip install wget
import os
import wget
from IPython.display import clear_output

#install diffusers for txt2image generation
!pip install --upgrade diffusers[torch] torch
clear_output()
print("\nSuccessfully installed diffusers[torch]\n")

#install diffusers requirements
!pip install --upgrade diffusers transformers accelerate safetensors OmegaConf
clear_output()
print("\nInstalled txt2image requirements\n")

#install interface requirements
!pip install gradio requests
clear_output()
print("\nInstalled interface requirements\n")

#Install diffusers
if not os.path.exists("/content/diffusers"):
  !mkdir /content/diffusers
  %cd /diffusers
  !git clone https://github.com/huggingface/diffusers
  !git config --global user.email "you@example.com"
  %cd /content/diffusers
  !git checkout 8a3f0c1f7178f4a3d5a5b21ae8c2906f473e240d
  clear_output()
  print("\nSuccesfully installed Diffusers\n")
  %cd /content
else:
  print("\nDiffusers alredy installed\n")

#Clone project repository
if not os.path.exists("/content/adventure"):
  !mkdir /content/adventure
  %cd /content/adventure
  !git clone https://github.com/Y0z64/Mythmaker
  clear_output()
  print("\nCorrectly cloned repository")
  %cd /content
else:
  print("\nRepository already cloned\n")

#Install model
if not os.path.exists("/content/stable_diffusion"):
  !mkdir /content/stable_diffusion
  %cd /content/stable_diffusion
  !wget -O protogen_x34.safetensors "https://civitai.com/api/download/models/4048?type=Model&format=SafeTensor"
  clear_output()
  print("\nCorrectly installed model\n")
else:
  clear_output()

clear_output()
print("Requirements correctly installed")
%cd /content


Installed interface requirements

[Errno 2] No such file or directory: '/diffusers'
/content
Cloning into 'diffusers'...
remote: Enumerating objects: 17868, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 17868 (delta 157), reused 164 (delta 77), pack-reused 17563
Receiving objects: 100% (17868/17868), 11.92 MiB | 27.99 MiB/s, done.
Resolving deltas: 100% (12399/12399), done.


###**Convert model to diffusers**

In [ ]:
import os
from IPython.display import clear_output
import sys

sys.path.append("/usr/local/lib/python3.8/dist-packages")


if not os.path.exists("/content/stable_diffusion/model"):
  %cd /content/stable_diffusion
  !mkdir model
  !python /content/diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py --from_safetensors --checkpoint_path /content/stable_diffusion/protogen_x34.safetensors --dump_path /content/stable_diffusion/model
  
  print("Correctly converted model to Diffusers\n")
  if not os.path.exists("/content/stable_diffusion/model"):
    !rm /content/stable_diffusion/protogen_x34.safetensors

else:
  clear_output()
  print("Model already converted")

%cd /content

Model already converted
/content


##**Start app**
When the link appears click it to run it in your browser.

In [ ]:

#add app to system path
import sys

#!cat /content/adventure/AI21-hackathon/adventures.py
sys.path.append("/content/adventure/AI21-hackathon")
!cd /content/adventure/AI21-hackathon
!pwd
from adventures import *
print("Custom app was imported correctly")


import torch
from diffusers import StableDiffusionPipeline
from IPython.display import Image
from time import sleep
import gradio as gr
from IPython.display import clear_output
#import app

#Initialize image generator with custom pretrained model
pipe = StableDiffusionPipeline.from_pretrained("/content/stable_diffusion/model", torch_dtype=torch.float16)
pipe = pipe.to("cuda")
generator = torch.Generator("cuda")

#Get API KEY
api_key = input("Input your API key: ")

#start the adventure
action_list, default_prpt = start_chat(fantasy_preset, start_1, api_key)

def display_chatbot(input_message, Temperature, hystory=[]):
    #call the chatbot
    history, history, actions, message = chatbot_requests(input_message, action_list, default_prpt, Temperature, api_key, hystory)

    #generate image
    generation_prompt = generate_prompt(message, api_key)
    image = pipe(generation_prompt, guidance_scale=7.5, num_inference_steps=15, generator=generator).images[0]

    return history, history, actions, image

#end adventure
def clear_save():
    global save
    save = []

#Interface design
demo = gr.Interface(fn=display_chatbot,
        inputs=["text", gr.Slider(0, 1), "state"],
        outputs=["chatbot", "state", "text", "image"])

#launch interface
demo.launch(debug=True, share=True)

#close everyhting
clear_output()

/content
Custom app was imported correctly


OSError: ignored